<a href="https://colab.research.google.com/github/johnathanacortesd/Diplomado_Ciencia_Datos/blob/main/Proyecto_Zero_shot_clasificador_noticias_SSTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Zero-shot calsificación de textos SSTuing

Instalar las dependencias.

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.7 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch, string, random

# Cargar el modelo y tokenizar
Para tareas en inglés se puede usar alguno de los siguientes modelos:
* DAMO-NLP-SG/zero-shot-classify-SSTuning-base
* DAMO-NLP-SG/zero-shot-classify-SSTuning-base
* DAMO-NLP-SG/zero-shot-classify-SSTuning-base

para tareas que no son en inglés usar el modelo
* DAMO-NLP-SG/zero-shot-classify-SSTuning-XLM-R

In [ ]:
model_name = "DAMO-NLP-SG/zero-shot-classify-SSTuning-XLM-R" # @param ["DAMO-NLP-SG/zero-shot-classify-SSTuning-base", "DAMO-NLP-SG/zero-shot-classify-SSTuning-large", "DAMO-NLP-SG/zero-shot-classify-SSTuning-ALBERT", "DAMO-NLP-SG/zero-shot-classify-SSTuning-XLM-R"]

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(model_name)

## Crea algunas funciones auxiliares para procesar los datos.


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
list_ABC = [x for x in string.ascii_uppercase]

def add_prefix(text, list_label, shuffle=False):
    # Append a period '.' to each label. This will improve the accuracy
    list_label = [x+'.' if x[-1] not in ['.','!'] else x for x in list_label]

    # Extend the list_label with padding tokens to have a length of 20
    list_label_new = list_label + [tokenizer.pad_token]* (20 - len(list_label))

    if shuffle:
        # Shuffle the order of elements in list_label_new if shuffle flag is True
        random.shuffle(list_label_new)

    # Create a string representation of label options by combining each label with its corresponding index
    s_option = ' '.join(['('+list_ABC[i]+') '+list_label_new[i] for i in range(len(list_label_new))])

    # Return the modified text with label options and the list_label_new
    return f'{s_option} {tokenizer.sep_token} {text}', list_label_new

In [ ]:
def check_text(model, text, list_label, shuffle=False):
    # Add prefixes to the text using the add_prefix function
    text, list_label_new = add_prefix(text, list_label, shuffle=shuffle)
    print('input text:   ', text)

    # Set the model to evaluation mode and move it to the appropriate device
    model.to(device).eval()

    # Perform tokenization and encoding of the text
    encoding = tokenizer([text], truncation=True, max_length=512)

    # Create a dictionary of tensors for the encoded text
    item = {key: torch.tensor(val).to(device) for key, val in encoding.items()}

    # Generate logits from the model
    logits = model(**item).logits

    # Select a subset of logits based on shuffle flag
    logits = logits if shuffle else logits[:, 0:len(list_label)]

    # Convert logits to probabilities using softmax
    probs = torch.nn.functional.softmax(logits, dim=-1).tolist()

    # Get the predicted label index
    predictions = torch.argmax(logits, dim=-1).item()

    # Round the probabilities to five decimal places
    probabilities = [round(x, 5) for x in probs[0]]

    print('probabilities:',probabilities)
    print(f'prediction:    {predictions} => ({list_ABC[predictions]}) {list_label_new[predictions]}')
    print(f'probability:   {round(probabilities[predictions]*100,2)}%')

# Inferencia

## Análisis de sentimiento

Proporcione la entrada y la lista de etiquetas.
Puede utilizar etiquetas originales o convertir las etiquetas en oraciones.

In [ ]:
text = "Como respuesta al trino del presidente Gustavo Petro sobre los cambios en la estructura del mercado de la cocaína, el exministro de educación Alejandro Gaviria le hizo una crítica a los planteamientos del mandatario en su página web"

list_label = ["Negativo","Positvo","Neutro"]
# list_label = ["It's terrible.","It's great."]

Procesar para generar una inferencia

In [ ]:
check_text(model,text,list_label, shuffle=False)

input text:    (A) Negativo. (B) Positvo. (C) Neutro. (D) <pad> (E) <pad> (F) <pad> (G) <pad> (H) <pad> (I) <pad> (J) <pad> (K) <pad> (L) <pad> (M) <pad> (N) <pad> (O) <pad> (P) <pad> (Q) <pad> (R) <pad> (S) <pad> (T) <pad> </s> Como respuesta al trino del presidente Gustavo Petro sobre los cambios en la estructura del mercado de la cocaína, el exministro de educación Alejandro Gaviria le hizo una crítica a los planteamientos del mandatario en su página we
probabilities: [0.69814, 0.1897, 0.11216]
prediction:    0 => (A) Negativo.
probability:   69.81%


## Clasificación de temas

Darle al texto una lista de temas



In [ ]:
text = "La recién renovada Jeep Renegade que llega al país, además de contar con características de rendimiento y performance que brinda su conjunto mecánico con motor 1.3L Turbo, generador de 173 hp y 270 Nm, viene equipada con un paquete de seguridad activa y pasiva, reforzado con tecnologías que brindan protección a sus ocupantes."

# list_label = ["politics","sports","business","technology"]
list_label = [
    "Activaciones locales de marca",
    "Deportes",
    "Diseño",
    "Electrificación",
    "Escasez inventarios - componentes",
    "Estadísticas – Mercado",
    "Eventos",
    "Fábricas – Producción",
    "Indefinido",
    "Influencers – aliados",
    "Investigación y Desarrollo",
    "Judiciales",
    "Lanzamientos",
    "Mención",
    "Pandemia",
    "Premios y reconocimientos",
    "Proyecciones - Balances",
    "Recall – Fallas técnicas"
]


In [ ]:
check_text(model,text,list_label, shuffle=False)

input text:    (A) Activaciones locales de marca. (B) Deportes. (C) Diseño. (D) Electrificación. (E) Escasez inventarios - componentes. (F) Estadísticas – Mercado. (G) Eventos. (H) Fábricas – Producción. (I) Indefinido. (J) Influencers – aliados. (K) Investigación y Desarrollo. (L) Judiciales. (M) Lanzamientos. (N) Mención. (O) Pandemia. (P) Premios y reconocimientos. (Q) Proyecciones - Balances. (R) Recall – Fallas técnicas. (S) <pad> (T) <pad> </s> La recién renovada Jeep Renegade que llega al país, además de contar con características de rendimiento y performance que brinda su conjunto mecánico con motor 1.3L Turbo, generador de 173 hp y 270 Nm, viene equipada con un paquete de seguridad activa y pasiva, reforzado con tecnologías que brindan protección a sus ocupantes.
probabilities: [0.25821, 0.01019, 0.07761, 0.05075, 0.01817, 0.05749, 0.01644, 0.05202, 0.05121, 0.03236, 0.12592, 0.00382, 0.04422, 0.06096, 0.01599, 0.07173, 0.01138, 0.04151]
prediction:    0 => (A) Activaciones lo


## Clasificación de sentimiento

In [ ]:
# Spanish
text = "La recién renovada Jeep Renegade que llega al país, además de contar con características de rendimiento y performance que brinda su conjunto mecánico con motor 1.3L Turbo, generador de 173 hp y 270 Nm, viene equipada con un paquete de seguridad activa y pasiva, reforzado con tecnologías que brindan protección a sus ocupantes."
list_label = ["Es positivo.","Es neutro.", "Es indefinido"]

check_text(model,text,list_label, shuffle=False)

input text:    (A) Es positivo. (B) Es neutro. (C) Es indefinido. (D) <pad> (E) <pad> (F) <pad> (G) <pad> (H) <pad> (I) <pad> (J) <pad> (K) <pad> (L) <pad> (M) <pad> (N) <pad> (O) <pad> (P) <pad> (Q) <pad> (R) <pad> (S) <pad> (T) <pad> </s> La recién renovada Jeep Renegade que llega al país, además de contar con características de rendimiento y performance que brinda su conjunto mecánico con motor 1.3L Turbo, generador de 173 hp y 270 Nm, viene equipada con un paquete de seguridad activa y pasiva, reforzado con tecnologías que brindan protección a sus ocupantes.
probabilities: [0.59359, 0.04823, 0.35818]
prediction:    0 => (A) Es positivo.
probability:   59.36%



## Prueba con noticias de la SAE

In [ ]:
!pip install transformers

import pandas as pd
import torch
import string
import random
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm

# Cargar el archivo XLSX desde el almacenamiento de sesión
from google.colab import files
uploaded = files.upload()
EXCEL_FILENAME = next(iter(uploaded))
EXCEL_PATH = './' + EXCEL_FILENAME

# Leer el archivo Excel
df = pd.read_excel(EXCEL_PATH)

# Cargar el modelo y el tokenizador
model_name = "DAMO-NLP-SG/zero-shot-classify-SSTuning-XLM-R"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
list_ABC = [x for x in string.ascii_uppercase]

# Función para agregar prefijos
def add_prefix(text, list_label, shuffle=False):
    list_label = [x + '.' if x[-1] not in ['.', '!'] else x for x in list_label]
    list_label_new = list_label + [tokenizer.pad_token] * (20 - len(list_label))

    if shuffle:
        random.shuffle(list_label_new)

    s_option = ' '.join(['(' + list_ABC[i] + ') ' + list_label_new[i] for i in range(len(list_label_new))])
    return f'{s_option} {tokenizer.sep_token} {text}', list_label_new

# Función para procesar el texto y predecir temas
def check_text(model, text, list_label, shuffle=False):
    text, list_label_new = add_prefix(text, list_label, shuffle=shuffle)

    model.to(device).eval()
    encoding = tokenizer([text], truncation=True, max_length=512)
    item = {key: torch.tensor(val).to(device) for key, val in encoding.items()}
    logits = model(**item).logits
    logits = logits if shuffle else logits[:, 0:len(list_label)]
    probs = torch.nn.functional.softmax(logits, dim=-1).tolist()[0]

    top_label_indices = torch.argsort(logits, descending=True).tolist()[0]
    top_labels = [list_label_new[i] for i in top_label_indices]
    top_probabilities = [probs[i] for i in top_label_indices]

    return top_labels, top_probabilities

# Realizar inferencia para cada resumen en el DataFrame con barra de progreso
results = []
for index, row in tqdm(df.iterrows(), total=len(df), desc="Procesando resúmenes"):
    text = row['Resumen']
    list_label = [
        "Extinción de dominio",
        "René Higuita",
        "Fiscalía",
        "Triple A",
        "Ecopetrol",
        "Cooperativas",
        "Reforma agraria",
        "Villavicencio",
        "Firmantes de paz",
        "Bienestar animal",
        "Opinión",
        "Hogar de paso y recuperación nutricional",
        "Respaldo Iván Velásquez",
        "Contraloría",
        "Predio de Mamonal",
        "Propuesta MinTrabajo",
        "Drogas La Rebaja",
        "Gasolina",
        "Depositarios",
        "Política de drogas",
        "Centro Democrático",
        "Asamblea Nacional de Economía Solidaria"
    ]
    top_labels, top_probabilities = check_text(model, text, list_label, shuffle=False)

    row_data = {
        "Resumen": text,
        "Temas Predichos": top_labels,
        "Prob Temas": top_probabilities,
    }
    results.append(row_data)

Saving sae.xlsx to sae.xlsx


Procesando resúmenes: 100%|██████████| 259/259 [04:19<00:00,  1.00s/it]


,Resumen,Temas Predichos,Prob Temas
0,En Colombia pululan y crecen como verdolagas l...,"[Extinción de dominio., Drogas La Rebaja., Bie...","[0.23610347509384155, 0.10516108572483063, 0.0..."
1,La Fundación Paz Animal nuevamente denunció qu...,"[Predio de Mamonal., Fiscalía., Bienestar anim...","[0.2916639745235443, 0.21061725914478302, 0.12..."
2,"A través de redes sociales, la organización de...","[Predio de Mamonal., Fiscalía., Bienestar anim...","[0.35947489738464355, 0.2226368635892868, 0.15..."
3,Pretendían distribuir más licor falsificado en...,"[Depositarios., Fiscalía., Propuesta MinTrabaj...","[0.3024807274341583, 0.16925448179244995, 0.07..."
4,En región con poca autoridad la organización r...,"[Depositarios., Fiscalía., Extinción de domini...","[0.181364044547081, 0.1747269630432129, 0.1224..."
...,...,...,...
254,Espartaco? Luis Jairo Ramírez H.Tbda la demago...,"[Extinción de dominio., Fiscalía., Opinión., P...","[0.21057483553886414, 0.1286286562681198, 0.09..."
255,"En una acción coordinada, las Fuerzas Militare...","[Fiscalía., Respaldo Iván Velásquez., Propuest...","[0.6782800555229187, 0.09443226456642151, 0.06..."
256,"En una acción coordinada, las Fuerzas Militare...","[Fiscalía., Propuesta MinTrabajo., Respaldo Iv...","[0.6860390305519104, 0.07400087267160416, 0.05..."
257,Las propiedades se encuentran en Villavicencio...,"[Extinción de dominio., Fiscalía., Depositario...","[0.7543225884437561, 0.11566124856472015, 0.11..."


## Vista previa de los resultados

In [ ]:
# Mostrar solo las 3 etiquetas con la mayor probabilidad para cada resumen
df_results_short = df_results.copy()
df_results_short["Temas Predichos"] = df_results_short["Temas Predichos"].apply(lambda x: x[:3])
display(df_results_short.head(11))

,Resumen,Temas Predichos,Prob Temas
0,En Colombia pululan y crecen como verdolagas l...,"[Extinción de dominio., Drogas La Rebaja., Bie...","[0.23610347509384155, 0.10516108572483063, 0.0..."
1,La Fundación Paz Animal nuevamente denunció qu...,"[Predio de Mamonal., Fiscalía., Bienestar anim...","[0.2916639745235443, 0.21061725914478302, 0.12..."
2,"A través de redes sociales, la organización de...","[Predio de Mamonal., Fiscalía., Bienestar anim...","[0.35947489738464355, 0.2226368635892868, 0.15..."
3,Pretendían distribuir más licor falsificado en...,"[Depositarios., Fiscalía., Propuesta MinTrabajo.]","[0.3024807274341583, 0.16925448179244995, 0.07..."
4,En región con poca autoridad la organización r...,"[Depositarios., Fiscalía., Extinción de dominio.]","[0.181364044547081, 0.1747269630432129, 0.1224..."
5,Están ubicados en el Valle del Cauca del ciuda...,"[Extinción de dominio., Fiscalía., Propuesta M...","[0.6600219011306763, 0.23453035950660706, 0.01..."
6,Constituyeron empresas para dar apariencia de ...,"[Fiscalía., Depositarios., Extinción de dominio.]","[0.44110289216041565, 0.124773770570755, 0.113..."
7,"En promedio, según Gerardo Vega Medina, direct...","[Reforma agraria., Propuesta MinTrabajo., Ecop...","[0.6661789417266846, 0.2661464810371399, 0.014..."
8,De las 325.777 hectáreas totales de gestión de...,"[Reforma agraria., Depositarios., Propuesta Mi...","[0.6405999064445496, 0.11364424228668213, 0.06..."
9,Agregue a sus temas de interés CerrarAdministr...,"[Reforma agraria., Fiscalía., Opinión.]","[0.5280529260635376, 0.18632599711418152, 0.06..."


## Creación de un nuevo excel con los resultados

In [ ]:
# Crear un DataFrame con los resultados y guardar solo los 3 principales temas y sus probabilidades en un nuevo archivo Excel
df_results = pd.DataFrame(results)
df_results["Temas Predichos"] = df_results["Temas Predichos"].apply(lambda x: x[:3])  # Mantener solo los 3 principales temas
df_results["Prob Temas"] = df_results["Prob Temas"].apply(lambda x: x[:3])  # Mantener solo las probabilidades de los 3 principales temas
output_filename = "resultados_temas.xlsx"
df_results.to_excel(output_filename, index=False)
print(f"Resultados guardados en {output_filename}")

Resultados guardados en resultados_temas.xlsx


@inproceedings{acl23/SSTuning,
  author    = {Chaoqun Liu and
               Wenxuan Zhang and
               Guizhen Chen and
               Xiaobao Wu and
               Anh Tuan Luu and
               Chip Hong Chang and
               Lidong Bing},
  title     = {Zero-Shot Text Classification via Self-Supervised Tuning},
  booktitle = {Findings of the Association for Computational Linguistics: ACL 2023},
  year      = {2023},
  url       = {https://arxiv.org/abs/2305.11442},
}
